In [160]:
import numpy as np
import pandas as pd

path = '/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/jeju_review_sentiment.csv'
df = pd.read_csv(path)
df.head()

,label,score,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자
0,1,0.948258,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,여행,"연인・배우자, 아이"
1,1,0.881020,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,여행,연인・배우자
2,0,0.972762,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이"
3,1,0.977383,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,여행,연인・배우자
4,1,0.560018,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,"데이트, 여행",연인・배우자


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165410 entries, 0 to 165409
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   label       165410 non-null  int64  
 1   score       165410 non-null  float64
 2   name        165410 non-null  object 
 3   address     165410 non-null  object 
 4   NAVER_name  165410 non-null  object 
 5   category    165410 non-null  object 
 6   review_num  165410 non-null  object 
 7   reviews     165410 non-null  object 
 8   이용방법        165410 non-null  object 
 9   대기시간        165410 non-null  object 
 10  방문목적        165410 non-null  object 
 11  동행자         165410 non-null  object 
 12  tokens      165410 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 16.4+ MB


In [4]:
df.shape

(165410, 12)

In [5]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 41.1 MB/s eta 0:00:00


In [161]:
import json
import re
from konlpy.tag import Okt
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud

reviews = df['reviews']
reviews

,reviews
0,제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...
1,오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...
2,순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...
3,"뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ..."
4,"제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실..."
...,...
165405,사진 찍을 겨를도 없이 맛있게 먹었네요^^.\n사장님이 친절하게 대해주셔서 자주 오...
165406,월남쌈 맛있어요
165407,맛있어요
165408,"양이 많아요,, 배불러서 좀 남겼는데 생각나네요ㅠ\n편백찜 + 샤브 + 칼국수 + ..."


In [79]:
okt = Okt()

# 명사 추출 함수
def extract_nouns(text):
    # 한글만 남기기 및 한글 자음(ㄱ-ㅎ), 모음(ㅏ-ㅣ)이 반복되는 부분 제거
    clean_text = re.sub(r'[^가-힣\s]', '', text)  # 특수문자 제거 (한글과 공백만 남기기)
    clean_text = re.sub(r'\s*[ㄱ-ㅎㅏ-ㅣ]+\s*', ' ', clean_text)  # ㅎㅎㅎ, ㅋㅋㅋ와 같은 자음만 있는 부분 제거
    # 명사만 추출
    nouns = okt.nouns(clean_text)
    return nouns

# 명사 리스트를 추출하여 'tokens' 열에 저장
df['tokens'] = df['reviews'].apply(extract_nouns)

In [80]:
df.head(5)

,label,score,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,tokens
0,1,0.948258,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,여행,"연인・배우자, 아이","[제주도, 여행, 처음, 갈치, 조림, 살이, 더, 먹기, 여행객, 월, 끝, 자락..."
1,1,0.881020,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,여행,연인・배우자,"[오픈, 런, 대기, 번, 사람, 생각, 식당, 안, 손님, 가득, 안쪽, 테이블,..."
2,0,0.972762,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이","[살, 갈치, 조림, 방문, 현장, 테이블, 링, 회전, 편이, 평일, 점심, 기준..."
3,1,0.977383,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,여행,연인・배우자,"[뼈, 가시, 갈치, 조림, 살, 갈치, 먹기, 젓가락, 필요, 갈치, 감자, 무,..."
4,1,0.560018,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,"데이트, 여행",연인・배우자,"[제주, 때, 시간, 재료, 진일, 때, 늘, 이번, 쯤, 대기, 시간, 분, 대기..."


In [81]:
df['tokens']

,tokens
0,"[제주도, 여행, 처음, 갈치, 조림, 살이, 더, 먹기, 여행객, 월, 끝, 자락..."
1,"[오픈, 런, 대기, 번, 사람, 생각, 식당, 안, 손님, 가득, 안쪽, 테이블,..."
2,"[살, 갈치, 조림, 방문, 현장, 테이블, 링, 회전, 편이, 평일, 점심, 기준..."
3,"[뼈, 가시, 갈치, 조림, 살, 갈치, 먹기, 젓가락, 필요, 갈치, 감자, 무,..."
4,"[제주, 때, 시간, 재료, 진일, 때, 늘, 이번, 쯤, 대기, 시간, 분, 대기..."
...,...
165405,"[사진, 찍, 겨를, 사장, 대해, 자주, 년]"
165406,[월남쌈]
165407,[]
165408,"[양, 좀, 편백, 찜, 샤브, 칼국수, 죽, 모두]"


In [82]:
#165407 index 데이터 불러오기
df.iloc[0]['tokens']

['제주도',
 '여행',
 '처음',
 '갈치',
 '조림',
 '살이',
 '더',
 '먹기',
 '여행객',
 '월',
 '끝',
 '자락',
 '건',
 '시간대',
 '분도',
 '안',
 '수',
 '성게',
 '미역국',
 '조금',
 '밍밍',
 '느낌',
 '살짝',
 '아이']

In [11]:
df.iloc[0]['reviews']

'제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이 많지 않은 9월의 끝자락이라 그런건지 시간대를 잘 맞춘건지 10분도 안 기다리고 들어갈 수 있어서 좋았어요~~\n성게미역국은 조금 밍밍한 느낌이라 살짝 아쉬웠지만 아이들은 맛있게 먹었습니다😊'

In [83]:
#불용어 제거
def remove_stopword(tokens, stopwords):
    filtered_tokens = [word for word in tokens if word not in stopwords and len(word) > 1]
    return filtered_tokens

stopwords = ['제주도', '제주']
df['tokens'] = df['tokens'].apply(lambda tokens: remove_stopword(tokens, stopwords))

In [84]:
df.head(5)

,label,score,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,tokens
0,1,0.948258,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,여행,"연인・배우자, 아이","[여행, 처음, 갈치, 조림, 살이, 먹기, 여행객, 자락, 시간대, 분도, 성게,..."
1,1,0.881020,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,여행,연인・배우자,"[오픈, 대기, 사람, 생각, 식당, 손님, 가득, 안쪽, 테이블, 현장, 캐치, ..."
2,0,0.972762,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이","[갈치, 조림, 방문, 현장, 테이블, 회전, 편이, 평일, 점심, 기준, 주차장,..."
3,1,0.977383,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,여행,연인・배우자,"[가시, 갈치, 조림, 갈치, 먹기, 젓가락, 필요, 갈치, 감자, 뚝딱, 갈치, ..."
4,1,0.560018,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,"데이트, 여행",연인・배우자,"[시간, 재료, 진일, 이번, 대기, 시간, 대기, 근처, 소품, 바로, 순금, 해..."


In [31]:
df.columns

Index(['label', 'score', 'name', 'address', 'NAVER_name', 'category',
       'review_num', 'reviews', '이용방법', '대기시간', '방문목적', '동행자', 'tokens'],
      dtype='object')

In [85]:
df.to_csv('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/jeju_review_keyword.csv', index=0)

In [162]:
df = pd.read_csv('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/jeju_review_keyword.csv')
df.head()

,label,score,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,tokens
0,1,0.948258,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,예약 없이 이용,10분 이내,여행,"연인・배우자, 아이","['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자..."
1,1,0.881020,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",오픈런 했고 대기 1번이라서 사람이 많이 없을거라 생각했는데 식당 안에 손님이 가득...,예약 후 이용,10분 이내,여행,연인・배우자,"['오픈', '대기', '사람', '생각', '식당', '손님', '가득', '안쪽..."
2,0,0.972762,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,예약 후 이용,30분 이내,여행,"연인・배우자, 아이","['갈치', '조림', '방문', '현장', '테이블', '회전', '편이', '평..."
3,1,0.977383,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","뼈없는, 가시없는 갈치조림먹으려고 찾아보고왔어요!\n순살갈치라니ㅋㅋ 먹기너무편하고 ...",예약 없이 이용,30분 이내,여행,연인・배우자,"['가시', '갈치', '조림', '갈치', '먹기', '젓가락', '필요', '갈..."
4,1,0.560018,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118","제주 올때마다 가보고 싶었는데 시간이 안맞기도 했고, 재료소진일 때도 있어서 늘 실...",예약 후 이용,1시간 이상,"데이트, 여행",연인・배우자,"['시간', '재료', '진일', '이번', '대기', '시간', '대기', '근처..."


In [88]:
# 컬럼 값을 리스트로 변환하는 함수
def to_list(value):
    return value.split(', ') if isinstance(value, str) else []

# 리스트로 변환
df['reviews'] = df['reviews'].apply(to_list)
df['이용방법'] = df['이용방법'].apply(to_list)
df['대기시간'] = df['대기시간'].apply(to_list)
df['방문목적'] = df['방문목적'].apply(to_list)
df['동행자'] = df['동행자'].apply(to_list)

# 그룹화하여 리스트 합치기
df_grouped = df.groupby(['name', 'address', 'NAVER_name', 'category', 'review_num'], as_index=False).agg({
    'reviews': lambda x: sum(x, []),
    '이용방법': lambda x: sum(x, []),
    '대기시간': lambda x: sum(x, []),
    '방문목적': lambda x: sum(x, []),
    '동행자': lambda x: sum(x, [])
})

In [89]:
df_grouped

,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자
0,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",[제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객...,"[예약 없이 이용, 예약 후 이용, 예약 후 이용, 예약 없이 이용, 예약 후 이용...","[10분 이내, 10분 이내, 30분 이내, 30분 이내, 1시간 이상, 30분 이...","[여행, 여행, 여행, 여행, 데이트, 여행, 데이트, 여행, 데이트, 데이트, 여...","[연인・배우자, 아이, 연인・배우자, 연인・배우자, 아이, 연인・배우자, 연인・배우..."
1,번영가든,제주 제주시 봉개동 1385번지 1층,번영가든,한식,14,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,[],[],[],[]
2,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,"[김밥 맛있어요, 일가면서 가끔 들리는 장소 맛도짱 입니다., 소고기2줄은 밖에서 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 포장·배달...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장]","[일상, 나들이, 일상, 일상]","[지인・동료, 연인・배우자, 지인・동료, 연인・배우자]"
3,번영로뷔페,제주 제주시 도련일동 2574-1번지,번영로뷔페,한식뷔페,1,[오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되...,[예약 없이 이용],[10분 이내],[일상],[혼자]
4,번영로휴게소,제주 제주시 봉개동 545번지,번영로휴게소,일반휴게소,52,"[굿, 좋아요, 새벽에 지나가다 들렀는데 좋네요, 좋아요~, 사이프러스로 골프 치러...",[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...
2336,인챠이,제주 제주시 이도이동 1987-2번지,인챠이,중식당,82,[해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[일상, 데이트, 일상, 일상, 일상, 친목, 나들이, 데이트, 여행, 데이트, 일...","[친척・형제, 연인・배우자, 혼자, 혼자, 연인・배우자, 친구, 친구, 혼자, 연인..."
2337,인터포레스트,제주 제주시 구좌읍 송당리 2092번지 2층,인터포레스트,"카페,디저트",631,"[어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[여행, 여행, 데이트, 여행, 여행, 데이트, 데이트, 데이트, 나들이, 여행, ...","[부모님, 연인・배우자, 혼자, 연인・배우자, 연인・배우자, 연인・배우자, 반려동물..."
2338,인풍,제주 제주시 건입동 1263-3번지,인풍감자탕,"찌개,전골",17,"[같이 가는곳, 뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박...","[예약 없이 이용, 예약 없이 이용]","[바로 입장, 바로 입장]","[일상, 일상]","[친구, 친구, 혼자]"
2339,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,"[""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 후 이용, 예약 없이 ...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[데이트, 친목, 친목, 일상, 일상, 데이트, 가족모임, 데이트, 일상, 데이트,...","[연인・배우자, 친구, 친구, 연인・배우자, 아이, 연인・배우자, 연인・배우자, 연..."


In [90]:
df_grouped.shape

(2341, 10)

In [91]:
df.groupby('name')['reviews'].count()

,reviews
name,
번네식당,100
번영가든,14
번영김밥,39
번영로뷔페,1
번영로휴게소,52
...,...
인챠이,82
인터포레스트,100
인풍,16


In [92]:
df.groupby('name')['label'].value_counts().unstack(fill_value=0)

label,0,1
name,,
번네식당,2,98
번영가든,1,13
번영김밥,5,34
번영로뷔페,0,1
번영로휴게소,2,50
...,...,...
인챠이,8,74
인터포레스트,0,100
인풍,1,15


In [93]:
result = df.groupby(['name', 'address', 'NAVER_name', 'category', 'review_num'])[['reviews','tokens']].sum().reset_index()
result

,name,address,NAVER_name,category,review_num,reviews,tokens
0,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",[제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객...,"['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자..."
1,번영가든,제주 제주시 봉개동 1385번지 1층,번영가든,한식,14,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작..."
2,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,"[김밥 맛있어요, 일가면서 가끔 들리는 장소 맛도짱 입니다., 소고기2줄은 밖에서 ...","['김밥']['가면', '가끔', '장소']['소고기']['사진', '실물', '매..."
3,번영로뷔페,제주 제주시 도련일동 2574-1번지,번영로뷔페,한식뷔페,1,[오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되...,"['오늘', '근처', '뷔페', '소개', '오픈', '개월', '인지', '일단..."
4,번영로휴게소,제주 제주시 봉개동 545번지,번영로휴게소,일반휴게소,52,"[굿, 좋아요, 새벽에 지나가다 들렀는데 좋네요, 좋아요~, 사이프러스로 골프 치러...","[][]['새벽'][]['사이프러스', '골프', '김밥', '간식', '거리', ..."
...,...,...,...,...,...,...,...
2336,인챠이,제주 제주시 이도이동 1987-2번지,인챠이,중식당,82,[해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁...,"['해물', '쟁반짜장', '돼지', '짜장면', '미니', '탕수육', '쟁반짜장..."
2337,인터포레스트,제주 제주시 구좌읍 송당리 2092번지 2층,인터포레스트,"카페,디저트",631,"[어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 ...","['어머니', '이모', '모시', '여행', '잠시', '무엇', '사장', '정..."
2338,인풍,제주 제주시 건입동 1263-3번지,인풍감자탕,"찌개,전골",17,"[같이 가는곳, 뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박...","[]['해장국', '자극', '막걸리', '천원', '대박'][]['건입동', '감..."
2339,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,"[""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 ...","['걱정', '고기', '야채', '불리', '금구', '취향', '댕댕', '캐리..."


In [94]:
result.shape

(2341, 7)

In [167]:
# 리뷰 데이터를 리스트로 결합하는 함수
def join_reviews_as_list(reviews):
    return list(reviews)

# label이 1인 데이터를 그룹화하고, reviews를 리스트로 결합
positive_tokens = df[df['label'] == 1].groupby('name').agg({
    'reviews': sum,  # 리뷰는 리스트로 합침
    'tokens': sum,  # tokens는 리스트로 합침
    'label': 'sum'  # label 값도 합침
}).reset_index()

# 결과 출력
positive_tokens[['name', 'reviews', 'tokens']]

<ipython-input-167-eb2f5b1aba36>:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  positive_tokens = df[df['label'] == 1].groupby('name').agg({
<ipython-input-167-eb2f5b1aba36>:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  positive_tokens = df[df['label'] == 1].groupby('name').agg({


,name,reviews,tokens
0,번네식당,제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,"['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자..."
1,번영가든,포장했는데 맛있었어용가격대비 좋아요좋아요~^^주차도 편하고 음식맛도있어요맛있게 먹었...,"['포장']['가격', '대비'][]['주차', '음식'][]['음식'][][]['..."
2,번영김밥,"김밥 맛있어요일가면서 가끔 들리는 장소 맛도짱 입니다.소고기2줄은 밖에서 먹고,멸추...","['김밥']['가면', '가끔', '장소']['소고기']['사진', '실물', '매..."
3,번영로뷔페,오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되었...,"['오늘', '근처', '뷔페', '소개', '오픈', '개월', '인지', '일단..."
4,번영로휴게소,굿좋아요새벽에 지나가다 들렀는데 좋네요좋아요~사이프러스로 골프 치러 가는 길에 김밥...,"[][]['새벽'][]['사이프러스', '골프', '김밥', '간식', '거리', ..."
...,...,...,...
2324,인챠이,해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁반...,"['해물', '쟁반짜장', '돼지', '짜장면', '미니', '탕수육', '쟁반짜장..."
2325,인터포레스트,"어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 좋...","['어머니', '이모', '모시', '여행', '잠시', '무엇', '사장', '정..."
2326,인풍,같이 가는곳뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박입니다...,"[]['해장국', '자극', '막걸리', '천원', '대박'][]['건입동', '감..."
2327,인화계,"""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 배...","['걱정', '고기', '야채', '불리', '금구', '취향', '댕댕', '캐리..."


In [164]:
positive_tokens = df[df['label'] == 1].groupby('name')[['reviews','tokens','label']].sum().reset_index()
positive_tokens

,name,reviews,tokens,label
0,번네식당,제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,"['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자...",98
1,번영가든,포장했는데 맛있었어용가격대비 좋아요좋아요~^^주차도 편하고 음식맛도있어요맛있게 먹었...,"['포장']['가격', '대비'][]['주차', '음식'][]['음식'][][]['...",13
2,번영김밥,"김밥 맛있어요일가면서 가끔 들리는 장소 맛도짱 입니다.소고기2줄은 밖에서 먹고,멸추...","['김밥']['가면', '가끔', '장소']['소고기']['사진', '실물', '매...",34
3,번영로뷔페,오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되었...,"['오늘', '근처', '뷔페', '소개', '오픈', '개월', '인지', '일단...",1
4,번영로휴게소,굿좋아요새벽에 지나가다 들렀는데 좋네요좋아요~사이프러스로 골프 치러 가는 길에 김밥...,"[][]['새벽'][]['사이프러스', '골프', '김밥', '간식', '거리', ...",50
...,...,...,...,...
2324,인챠이,해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁반...,"['해물', '쟁반짜장', '돼지', '짜장면', '미니', '탕수육', '쟁반짜장...",74
2325,인터포레스트,"어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 좋...","['어머니', '이모', '모시', '여행', '잠시', '무엇', '사장', '정...",100
2326,인풍,같이 가는곳뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박입니다...,"[]['해장국', '자극', '막걸리', '천원', '대박'][]['건입동', '감...",15
2327,인화계,"""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 배...","['걱정', '고기', '야채', '불리', '금구', '취향', '댕댕', '캐리...",41


In [151]:
positive_tokens['label'] = 1
positive_tokens

,name,reviews,tokens,label
0,번네식당,[제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객...,"['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자...",1
1,번영가든,"[포장했는데 맛있었어용, 가격대비 좋아요, 좋아요~^^, 주차도 편하고 음식맛도있어...","['포장']['가격', '대비'][]['주차', '음식'][]['음식'][][]['...",1
2,번영김밥,"[김밥 맛있어요, 일가면서 가끔 들리는 장소 맛도짱 입니다., 소고기2줄은 밖에서 ...","['김밥']['가면', '가끔', '장소']['소고기']['사진', '실물', '매...",1
3,번영로뷔페,[오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되...,"['오늘', '근처', '뷔페', '소개', '오픈', '개월', '인지', '일단...",1
4,번영로휴게소,"[굿, 좋아요, 새벽에 지나가다 들렀는데 좋네요, 좋아요~, 사이프러스로 골프 치러...","[][]['새벽'][]['사이프러스', '골프', '김밥', '간식', '거리', ...",1
...,...,...,...,...
2324,인챠이,[해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁...,"['해물', '쟁반짜장', '돼지', '짜장면', '미니', '탕수육', '쟁반짜장...",1
2325,인터포레스트,"[어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 ...","['어머니', '이모', '모시', '여행', '잠시', '무엇', '사장', '정...",1
2326,인풍,"[같이 가는곳, 뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박...","[]['해장국', '자극', '막걸리', '천원', '대박'][]['건입동', '감...",1
2327,인화계,"[""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 ...","['걱정', '고기', '야채', '불리', '금구', '취향', '댕댕', '캐리...",1


In [152]:
negative_tokens = df[df['label'] == 0].groupby('name')[['reviews','tokens','label']].sum().reset_index()
negative_tokens

,name,reviews,tokens,label
0,번네식당,[순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 ...,"['갈치', '조림', '방문', '현장', '테이블', '회전', '편이', '평...",0
1,번영가든,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작...",0
2,번영김밥,"[아침 간식, 여전히 일가는 도중에 들려서 몇줄 샀어요., 참치, 소고기 김밥 샀는...","['아침', '간식']['일가', '도중']['참치', '소고기', '김밥', '오...",0
3,번영로휴게소,[오늘은 잔치국수와 김밥 먹었습니다. 땡초 김밥은 생각보다 매워서 빼고 먹었다는.....,"['오늘', '잔치국수', '김밥', '땡초', '김밥', '생각']['방아']",0
4,벌크커피 연북로점,"[벌크커피, 많이 주네요]",['커피']['주네'],0
...,...,...,...,...
1875,인정왕김밥,[동네 이웃이 추천해줘서 갔어요.\n제 입맛에는 별로였어요.\n즉석떡볶이는 너무 달...,"['동네', '이웃', '추천', '입맛', '별로', '떡볶이', '떡볶이', '...",0
1876,인챠이,[제주시 근처 중국집 찾아보고 왔습니다 리뷰도 괜찮고 찾아보니 나쁘지 않은것 같아 ...,"['제주시', '근처', '중국집', '리뷰', '칠리', '새우', '크림', '...",0
1877,인풍,[뼈다귀탕 먹었는데 맛집 보다는 가성비 좋은 집 ㅎ 술이 2500원],"['맛집', '가성']",0
1878,인화계,"[추천받아서 갔는데 간이 제 입에는 좀 짰어요, 닭갈비는 맛있어요~ 찌개는 좀......","['추천', '간이']['닭갈비', '찌개', '계란찜', '막국수'][]",0


In [153]:
negative_tokens['label'] = 0
negative_tokens

,name,reviews,tokens,label
0,번네식당,[순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 ...,"['갈치', '조림', '방문', '현장', '테이블', '회전', '편이', '평...",0
1,번영가든,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작...",0
2,번영김밥,"[아침 간식, 여전히 일가는 도중에 들려서 몇줄 샀어요., 참치, 소고기 김밥 샀는...","['아침', '간식']['일가', '도중']['참치', '소고기', '김밥', '오...",0
3,번영로휴게소,[오늘은 잔치국수와 김밥 먹었습니다. 땡초 김밥은 생각보다 매워서 빼고 먹었다는.....,"['오늘', '잔치국수', '김밥', '땡초', '김밥', '생각']['방아']",0
4,벌크커피 연북로점,"[벌크커피, 많이 주네요]",['커피']['주네'],0
...,...,...,...,...
1875,인정왕김밥,[동네 이웃이 추천해줘서 갔어요.\n제 입맛에는 별로였어요.\n즉석떡볶이는 너무 달...,"['동네', '이웃', '추천', '입맛', '별로', '떡볶이', '떡볶이', '...",0
1876,인챠이,[제주시 근처 중국집 찾아보고 왔습니다 리뷰도 괜찮고 찾아보니 나쁘지 않은것 같아 ...,"['제주시', '근처', '중국집', '리뷰', '칠리', '새우', '크림', '...",0
1877,인풍,[뼈다귀탕 먹었는데 맛집 보다는 가성비 좋은 집 ㅎ 술이 2500원],"['맛집', '가성']",0
1878,인화계,"[추천받아서 갔는데 간이 제 입에는 좀 짰어요, 닭갈비는 맛있어요~ 찌개는 좀......","['추천', '간이']['닭갈비', '찌개', '계란찜', '막국수'][]",0


In [154]:
merged_df = pd.merge(positive_tokens, negative_tokens, on='name', how='outer', suffixes=('_pos', '_neg'))

In [158]:
# NaN 값을 빈 리스트로 대체
merged_df['tokens_pos'] = merged_df['tokens_pos'].apply(lambda x: x if isinstance(x, list) else [])
merged_df['tokens_neg'] = merged_df['tokens_neg'].apply(lambda x: x if isinstance(x, list) else [])

# reviews_pos와 reviews_neg가 NaN일 때 빈 리스트로 표시



In [130]:
# label_pos와 label_neg 생성
merged_df['label_pos'] = 1
merged_df['label_neg'] = 0

In [159]:
merged_df

,name,reviews_pos,tokens_pos,label_pos,reviews_neg,tokens_neg,label_neg
0,번네식당,[제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객...,[],1.0,[순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 ...,[],0.0
1,번영가든,"[포장했는데 맛있었어용, 가격대비 좋아요, 좋아요~^^, 주차도 편하고 음식맛도있어...",[],1.0,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,[],0.0
2,번영김밥,"[김밥 맛있어요, 일가면서 가끔 들리는 장소 맛도짱 입니다., 소고기2줄은 밖에서 ...",[],1.0,"[아침 간식, 여전히 일가는 도중에 들려서 몇줄 샀어요., 참치, 소고기 김밥 샀는...",[],0.0
3,번영로뷔페,[오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되...,[],1.0,NaN,[],NaN
4,번영로휴게소,"[굿, 좋아요, 새벽에 지나가다 들렀는데 좋네요, 좋아요~, 사이프러스로 골프 치러...",[],1.0,[오늘은 잔치국수와 김밥 먹었습니다. 땡초 김밥은 생각보다 매워서 빼고 먹었다는.....,[],0.0
...,...,...,...,...,...,...,...
2325,인챠이,[해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁...,[],1.0,[제주시 근처 중국집 찾아보고 왔습니다 리뷰도 괜찮고 찾아보니 나쁘지 않은것 같아 ...,[],0.0
2326,인터포레스트,"[어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 ...",[],1.0,NaN,[],NaN
2327,인풍,"[같이 가는곳, 뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박...",[],1.0,[뼈다귀탕 먹었는데 맛집 보다는 가성비 좋은 집 ㅎ 술이 2500원],[],0.0
2328,인화계,"[""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 ...",[],1.0,"[추천받아서 갔는데 간이 제 입에는 좀 짰어요, 닭갈비는 맛있어요~ 찌개는 좀......",[],0.0


In [156]:
# 문자열에서 기호를 제거하고 리스트로 변환하는 함수
import re
# 문자열에서 기호를 제거하고 리스트로 변환하는 함수
def clean_and_convert_to_list(s):
    if pd.isnull(s):  # null값인 경우 빈 리스트 반환
        return []
    # 정규 표현식을 사용하여 기호 제거
    clean_string = re.sub(r"[\'\[\]]", "", s)  # '와 [ ] 제거
    # 쉼표로 분할하고 공백 제거 후 리스트로 변환
    return [item.strip() for item in clean_string.split(',') if item.strip()]

# 문자열을 리스트로 변환
merged_df['reviews_pos'] = merged_df['reviews_pos'].apply(clean_and_convert_to_list)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [132]:
merged_df.to_csv('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/jeju_review_keyword_merge.csv', index=0)

In [148]:
merged_df = pd.read_csv('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/jeju_review_keyword_merge.csv')
merged_df.head()

,name,reviews_pos,tokens_pos,label_pos,reviews_neg,tokens_neg,label_neg
0,번네식당,['제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행...,[],1,['순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른...,[],0
1,번영가든,"['포장했는데 맛있었어용', '가격대비 좋아요', '좋아요~^^', '주차도 편하고...",[],1,['식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골...,[],0
2,번영김밥,"['김밥 맛있어요', '일가면서 가끔 들리는 장소 맛도짱 입니다.', '소고기2줄은...",[],1,"['아침 간식', '여전히 일가는 도중에 들려서 몇줄 샀어요.', '참치', '소고...",[],0
3,번영로뷔페,['오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 ...,[],1,NaN,[],0
4,번영로휴게소,"['굿', '좋아요', '새벽에 지나가다 들렀는데 좋네요', '좋아요~', '사이프...",[],1,['오늘은 잔치국수와 김밥 먹었습니다. 땡초 김밥은 생각보다 매워서 빼고 먹었다는....,[],0


In [104]:
merged_df.shape

(2330, 7)

In [106]:
df_grouped['tokens'] = result['tokens']

In [107]:
df_grouped

,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,tokens
0,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",[제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객...,"[예약 없이 이용, 예약 후 이용, 예약 후 이용, 예약 없이 이용, 예약 후 이용...","[10분 이내, 10분 이내, 30분 이내, 30분 이내, 1시간 이상, 30분 이...","[여행, 여행, 여행, 여행, 데이트, 여행, 데이트, 여행, 데이트, 데이트, 여...","[연인・배우자, 아이, 연인・배우자, 연인・배우자, 아이, 연인・배우자, 연인・배우...","['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자..."
1,번영가든,제주 제주시 봉개동 1385번지 1층,번영가든,한식,14,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,[],[],[],[],"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작..."
2,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,"[김밥 맛있어요, 일가면서 가끔 들리는 장소 맛도짱 입니다., 소고기2줄은 밖에서 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 포장·배달...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장]","[일상, 나들이, 일상, 일상]","[지인・동료, 연인・배우자, 지인・동료, 연인・배우자]","['김밥']['가면', '가끔', '장소']['소고기']['사진', '실물', '매..."
3,번영로뷔페,제주 제주시 도련일동 2574-1번지,번영로뷔페,한식뷔페,1,[오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되...,[예약 없이 이용],[10분 이내],[일상],[혼자],"['오늘', '근처', '뷔페', '소개', '오픈', '개월', '인지', '일단..."
4,번영로휴게소,제주 제주시 봉개동 545번지,번영로휴게소,일반휴게소,52,"[굿, 좋아요, 새벽에 지나가다 들렀는데 좋네요, 좋아요~, 사이프러스로 골프 치러...",[],[],[],[],"[][]['새벽'][]['사이프러스', '골프', '김밥', '간식', '거리', ..."
...,...,...,...,...,...,...,...,...,...,...,...
2336,인챠이,제주 제주시 이도이동 1987-2번지,인챠이,중식당,82,[해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[일상, 데이트, 일상, 일상, 일상, 친목, 나들이, 데이트, 여행, 데이트, 일...","[친척・형제, 연인・배우자, 혼자, 혼자, 연인・배우자, 친구, 친구, 혼자, 연인...","['해물', '쟁반짜장', '돼지', '짜장면', '미니', '탕수육', '쟁반짜장..."
2337,인터포레스트,제주 제주시 구좌읍 송당리 2092번지 2층,인터포레스트,"카페,디저트",631,"[어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[여행, 여행, 데이트, 여행, 여행, 데이트, 데이트, 데이트, 나들이, 여행, ...","[부모님, 연인・배우자, 혼자, 연인・배우자, 연인・배우자, 연인・배우자, 반려동물...","['어머니', '이모', '모시', '여행', '잠시', '무엇', '사장', '정..."
2338,인풍,제주 제주시 건입동 1263-3번지,인풍감자탕,"찌개,전골",17,"[같이 가는곳, 뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박...","[예약 없이 이용, 예약 없이 이용]","[바로 입장, 바로 입장]","[일상, 일상]","[친구, 친구, 혼자]","[]['해장국', '자극', '막걸리', '천원', '대박'][]['건입동', '감..."
2339,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,"[""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 후 이용, 예약 없이 ...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[데이트, 친목, 친목, 일상, 일상, 데이트, 가족모임, 데이트, 일상, 데이트,...","[연인・배우자, 친구, 친구, 연인・배우자, 아이, 연인・배우자, 연인・배우자, 연...","['걱정', '고기', '야채', '불리', '금구', '취향', '댕댕', '캐리..."


In [119]:
# merge_df의 name 값이 df_grouped의 name에 있는 경우에만 concat
result = pd.merge(df_grouped, merged_df, on='name', how='inner')
result

,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,tokens,reviews_pos,tokens_pos,label_pos,reviews_neg,tokens_neg,label_neg
0,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",[제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객...,"[예약 없이 이용, 예약 후 이용, 예약 후 이용, 예약 없이 이용, 예약 후 이용...","[10분 이내, 10분 이내, 30분 이내, 30분 이내, 1시간 이상, 30분 이...","[여행, 여행, 여행, 여행, 데이트, 여행, 데이트, 여행, 데이트, 데이트, 여...","[연인・배우자, 아이, 연인・배우자, 연인・배우자, 아이, 연인・배우자, 연인・배우...","['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자...",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,"['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자...",1,순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,"['갈치', '조림', '방문', '현장', '테이블', '회전', '편이', '평...",0
1,번영가든,제주 제주시 봉개동 1385번지 1층,번영가든,한식,14,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,[],[],[],[],"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작...",포장했는데 맛있었어용가격대비 좋아요좋아요~^^주차도 편하고 음식맛도있어요맛있게 먹었...,"['포장', '가격', '대비', '주차', '음식', '음식', '식사', '반찬...",1,식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고루...,"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작...",0
2,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,"[김밥 맛있어요, 일가면서 가끔 들리는 장소 맛도짱 입니다., 소고기2줄은 밖에서 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 포장·배달...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장]","[일상, 나들이, 일상, 일상]","[지인・동료, 연인・배우자, 지인・동료, 연인・배우자]","['김밥']['가면', '가끔', '장소']['소고기']['사진', '실물', '매...","김밥 맛있어요일가면서 가끔 들리는 장소 맛도짱 입니다.소고기2줄은 밖에서 먹고,멸추...","['김밥', '가면', '가끔', '장소', '소고기', '사진', '실물', '매...",1,"아침 간식여전히 일가는 도중에 들려서 몇줄 샀어요.참치, 소고기 김밥 샀는데 맛있어...","['아침', '간식', '일가', '도중', '참치', '소고기', '김밥', '오...",0
3,번영로뷔페,제주 제주시 도련일동 2574-1번지,번영로뷔페,한식뷔페,1,[오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되...,[예약 없이 이용],[10분 이내],[일상],[혼자],"['오늘', '근처', '뷔페', '소개', '오픈', '개월', '인지', '일단...",오늘은 집근처에 있는 뷔페집 왔어요\n소개로 오게 되었는데\n오픈하신지 2개월 되었...,"['오늘', '근처', '뷔페', '소개', '오픈', '개월', '인지', '일단...",1,[],[],0
4,번영로휴게소,제주 제주시 봉개동 545번지,번영로휴게소,일반휴게소,52,"[굿, 좋아요, 새벽에 지나가다 들렀는데 좋네요, 좋아요~, 사이프러스로 골프 치러...",[],[],[],[],"[][]['새벽'][]['사이프러스', '골프', '김밥', '간식', '거리', ...",굿좋아요새벽에 지나가다 들렀는데 좋네요좋아요~사이프러스로 골프 치러 가는 길에 김밥...,"['새벽', '사이프러스', '골프', '김밥', '간식', '거리', '애용', ...",1,오늘은 잔치국수와 김밥 먹었습니다. 땡초 김밥은 생각보다 매워서 빼고 먹었다는..ㅎ...,"['오늘', '잔치국수', '김밥', '땡초', '김밥', '생각', '방아']",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336,인챠이,제주 제주시 이도이동 1987-2번지,인챠이,중식당,82,[해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[일상, 데이트, 일상, 일상, 일상, 친목, 나들이, 데이트, 여행, 데이트, 일...","[친척・형제, 연인・배우자, 혼자, 혼자, 연인・배우자, 친구, 친구, 혼자, 연인...","['해물', '쟁반짜장', '돼지', '짜장면', '미니', '탕수육', '쟁반짜장...",해물쟁반짜장 1인 9.0\n흑돼지짜장면(곱) 8.0\n미니탕수육 11.0\n\n쟁반...,"['해물', '쟁반짜장', '돼지', '짜장면', '미니', '탕수육', '쟁반짜장...",1,제주시 근처 중국집 찾아보고 왔습니다 리뷰도 괜찮고 찾아보니 나쁘지 않은것 같아 기...,"['제주시', '근처', '중국집', '리뷰', '칠리', '새우', '크림', '...",0
2337,인터포레스트,제주 제주시 구좌읍 송당리 2092번지 2층,인터포레스트,"카페,디저트",631,"[어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[여행, 여행, 데이트, 여행, 여행, 데이트, 데이트, 데이트, 나들이, 여행, ...","[부모님, 연인・배우자, 혼자, 연인・배우자, 연인・배우자, 연인・배우자, 반려동물...","['어머니', '이모', '모시', '여행', '잠시', '무엇', '사장', '정...","어머니, 이모님 모시고 여행중 잠시 쉬어가려 왔는데, 쾌적하고 아늑하고 너무너무 좋...","['어머니', '이모', '모시', '여행', '잠시', '무엇', '사장', '정...",1,[],[],0
2338,인풍,제주 제주시 건입동 1263-3번지,인풍감자탕,"찌개,전골",17,"[같이 가는곳, 뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박...","[예약 없이 이용, 예약 없이 이용]","[바로 입장, 바로 입장]","[일상, 일상]","[친구, 친구, 혼자]","[]['해장국', '자극', '막걸리', '천원', '대박'][]['건입동', '감...",같이 가는곳뼈다귀해장국 자극적이지 않고 얼큰하게 맛있어요. 막걸리 2천원 대박입니다...,"['해장국', '자극', '막걸리', '천원', '대박', '건입동', '감자탕',...",1,뼈다귀탕 먹었는데 맛집 보다는 가성비 좋은 집 ㅎ 술이 2500원,"['맛집', '가성']",0
2339,인화계,제주 제주시 일도이동 379-19번지 1층,인화계,닭갈비,46,"[""걱정마 안비싸"" ㅋㅋ\n고기도 부드럽고 , 야채도 싱싱하고\n싱싱하고넘 맛나게 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 후 이용, 예약 없이 ...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[데이트, 친목, 친목, 일상, 일상, 데이트, 가족모임, 데이트, 일상, 데이트,...","[연인・배우자, 친구, 친구, 연인・배우자, 아이, 

In [120]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341 entries, 0 to 2340
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         2341 non-null   object
 1   address      2341 non-null   object
 2   NAVER_name   2341 non-null   object
 3   category     2341 non-null   object
 4   review_num   2341 non-null   object
 5   reviews      2341 non-null   object
 6   이용방법         2341 non-null   object
 7   대기시간         2341 non-null   object
 8   방문목적         2341 non-null   object
 9   동행자          2341 non-null   object
 10  tokens       2341 non-null   object
 11  reviews_pos  2341 non-null   object
 12  tokens_pos   2341 non-null   object
 13  label_pos    2341 non-null   int64 
 14  reviews_neg  2341 non-null   object
 15  tokens_neg   2341 non-null   object
 16  label_neg    2341 non-null   int64 
dtypes: int64(2), object(15)
memory usage: 311.0+ KB


In [121]:
result.tail(20)

,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,tokens,reviews_pos,tokens_pos,label_pos,reviews_neg,tokens_neg,label_neg
2321,인생민물장어대정점,제주 서귀포시 대정읍 하모리 940-4번지 2층,인생민물장어대정점,한식,4,[너무너무 맛있습니다 사장님이 친절하시고 고향집에 온것 같은 편안한 곳입니다 가성비...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 후 이용]","[바로 입장, 바로 입장, 바로 입장, 바로 입장]","[나들이, 일상, 데이트, 가족모임]","[지인・동료, 연인・배우자, 아이, 부모님, 부모님, 부모님]","['사장', '고향', '가성', '비도', '인심', '지인', '추천', '모슬...",너무너무 맛있습니다 사장님이 친절하시고 고향집에 온것 같은 편안한 곳입니다 가성비도...,"['사장', '고향', '가성', '비도', '인심', '지인', '추천', '모슬...",1,[],[],0
2322,인생민물장어동홍점,제주 서귀포시 동홍동 114-11번지 1층,인생민물장어동홍점,한식,94,[장어구이정식 넘 알차네요. 점심특선인데 저녁에도 가능했어요.\n동홍동맛집 인생민물...,"[예약 없이 이용, 예약 후 이용, 예약 없이 이용, 예약 후 이용, 예약 없이 이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[데이트, 여행, 일상, 비즈니스, 기념일, 가족모임, 데이트, 나들이, 일상, 가...","[연인・배우자, 연인・배우자, 아이, 지인・동료, 연인・배우자, 아이, 연인・배우자...","['장어구이', '정식', '점심', '특선', '저녁', '동홍동', '맛집', ...",장어구이정식 넘 알차네요. 점심특선인데 저녁에도 가능했어요.\n동홍동맛집 인생민물장...,"['장어구이', '정식', '점심', '특선', '저녁', '동홍동', '맛집', ...",1,👍 👍 👍 👍 👍,[],0
2323,인서네,제주 제주시 도두일동 1249-9번지 1층,인서네,한식뷔페,29,[간판에 향토뷔페라고 되있어서 상호명은 ^_^; 영수증보고 알았네요. 이런 괜찮은 ...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용]","[바로 입장, 바로 입장, 바로 입장, 바로 입장]","[나들이, 여행, 나들이, 여행, 일상, 기념일, 일상, 회식]","[지인・동료, 연인・배우자, 아이, 부모님]","['간판', '향토', '뷔페', '상호', '영수증', '보고', '시장', '반...",간판에 향토뷔페라고 되있어서 상호명은 ^_^; 영수증보고 알았네요. 이런 괜찮은 시...,"['간판', '향토', '뷔페', '상호', '영수증', '보고', '시장', '반...",1,"념일 지인 인서네라는 가정식백반, 뷔페식당에 다녀 왔습니다.\n반찬이 매우 다양하며...","['념일', '지인', '가정식', '백반', '뷔페', '식당', '반찬', '매...",0
2324,인섬스토리2,제주 제주시 한림읍 협재리 3000번지 1층,인섬스토리2,한식,30,[려 비양도에서 우연히 발견한 보물같은 집!\n자전거 무료대여와 반려 동반 가능해서...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 10분 이내, 바로 입장, 바로 입장, 바로...","[여행, 나들이, 나들이, 일상, 여행, 여행, 여행, 여행, 나들이, 여행, 친목...","[부모님, 반려동물, 연인・배우자, 연인・배우자, 부모님, 친구, 친구, 부모님, ...","['비양도', '발견', '보물', '자전거', '무료', '대여', '반려', '...",려 비양도에서 우연히 발견한 보물같은 집!\n자전거 무료대여와 반려 동반 가능해서 ...,"['비양도', '발견', '보물', '자전거', '무료', '대여', '반려', '...",1,친절하시긴한데 메뉴 2개 주문했는데 하나 먹고있을 때 들어온 단체손님꺼 먼저 해준다...,"['메뉴', '주문', '하나', '단체', '손님', '먼저', '시간', '먹음...",0
2325,인손,제주 제주시 구좌읍 세화리 1448-1번지 1층,인손,요리주점,286,[제주사는 친구 소개로 (인생참치 1위) 방문했어요! 음식이 엄청 신선하고 맛있어요...,"[예약 없이 이용, 포장·배달 이용, 예약 없이 이용, 예약 없이 이용, 포장·배달...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 30분 이내, 바로 입장, 바로...","[친목, 여행, 여행, 친목, 여행, 데이트, 가족모임, 데이트, 여행, 데이트, ...","[연인・배우자, 친구, 혼자, 지인・동료, 연인・배우자, 아이, 연인・배우자, 혼자...","['주사', '친구', '소개', '인생', '참치', '방문', '음식', '광어...",제주사는 친구 소개로 (인생참치 1위) 방문했어요! 음식이 엄청 신선하고 맛있어요…...,"['주사', '친구', '소개', '인생', '참치', '방문', '음식', '광어...",1,한판에 26마리나 있었고 같이 포장해준건 사진대로와 오뎅국물 ? 새우머리는 따로 없...,"['한판', '마리나', '포장', '사진', '대로', '오뎅', '국물', '새...",0
2326,인수네정육식당,제주 제주시 구좌읍 김녕리 1633번지 구좌읍,인수네정육식당,한식,25,[숙소근처에서 점심먹으러 방문한 곳인데\n음식도 맛있고 친절하셨습니다~\n낮부터 고...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[여행, 가족모임, 데이트, 가족모임, 일상, 데이트, 데이트, 데이트, 데이트, ...","[연인・배우자, 연인・배우자, 연인・배우자, 연인・배우자, 혼자, 연인・배우자, 연...","['숙소', '근처', '점심', '방문', '음식', '기도', '그릇', '싹싹...",숙소근처에서 점심먹으러 방문한 곳인데\n음식도 맛있고 친절하셨습니다~\n낮부터 고기...,"['숙소', '근처', '점심', '방문', '음식', '기도', '그릇', '싹싹...",1,내장탕이 냄새가 나서 반도 못 먹었어요ㅠㅠ,"['장탕', '냄새', '반도']",0
2327,인스턴트버거,제주 제주시 이호일동 440-7번지,인스턴트버거,햄버거,670,[햄버거를 좋아해서 공항근처찾다가 오게됐는데\n진짜 인생버거임.. 엄청맛있음. 패티...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 포장·배달 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[여행, 일상, 여행, 여행, 데이트, 데이트, 일상, 여행, 여행, 데이트, 여행...","[연인・배우자, 연인・배우자, 부모님, 연인・배우자, 연인・배우자, 연인・배우자, ...","['햄버거', '공항', '근처', '진짜', '인생', '버거', '패티', '육...",햄버거를 좋아해서 공항근처찾다가 오게됐는데\n진짜 인생버거임.. 엄청맛있음. 패티가...,"['햄버거', '공항', '근처', '진짜', '인생', '버거', '패티', '육...",1,굳이 맥런치 시간에 여길 온다는 이유는? ☆☆☆☆☆\n그리고 ㅡ 감자튀김에 들어간 ...,"['굳이', '맥런치', '시간', '이유', '감자', '튀김', '후추', '와...",0
2328,인쌩맥주 제주시청점,제주 제주시 이도이동 1771-1번지 1층,인쌩맥주 제주시청점,"맥주,호프",30,[제가 가기에는 너무 젊은층이였지만\n그래도 좋은 경험을 하게 된 술집이네요ㅎㅎ\n...,"[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 ...","[친목, 일상, 친목, 친목, 데이트, 친목, 일상, 일상]","[지인・동료

In [122]:
result.drop(columns=['label_pos', 'label_neg'], inplace=True)

In [123]:
result.head(3)

,name,address,NAVER_name,category,review_num,reviews,이용방법,대기시간,방문목적,동행자,tokens,reviews_pos,tokens_pos,reviews_neg,tokens_neg
0,번네식당,제주 서귀포시 안덕면 화순리 1810-11번지 번네식당,번네식당,한식,"3,118",[제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객...,"[예약 없이 이용, 예약 후 이용, 예약 후 이용, 예약 없이 이용, 예약 후 이용...","[10분 이내, 10분 이내, 30분 이내, 30분 이내, 1시간 이상, 30분 이...","[여행, 여행, 여행, 여행, 데이트, 여행, 데이트, 여행, 데이트, 데이트, 여...","[연인・배우자, 아이, 연인・배우자, 연인・배우자, 아이, 연인・배우자, 연인・배우...","['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자...",제주도 여행 하면서 처음 먹어본 갈치조림! 순살이라서 더 먹기 편했어요^^ 여행객이...,"['여행', '처음', '갈치', '조림', '살이', '먹기', '여행객', '자...",순살 갈치 조림이 궁금해서 방문 했습니다\n현장 테이블링만 가능하고 회전은 빠른 편...,"['갈치', '조림', '방문', '현장', '테이블', '회전', '편이', '평..."
1,번영가든,제주 제주시 봉개동 1385번지 1층,번영가든,한식,14,[식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고...,[],[],[],[],"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작...",포장했는데 맛있었어용가격대비 좋아요좋아요~^^주차도 편하고 음식맛도있어요맛있게 먹었...,"['포장', '가격', '대비', '주차', '음식', '음식', '식사', '반찬...",식당에 들어서는 순간 가성비가 좋을 것 같은 기대감 상승해서 기분 좋아짐.\n골고루...,"['식당', '순간', '가성', '대감', '상승', '기분', '먹기', '시작..."
2,번영김밥,제주 제주시 봉개동 1902번지,번영김밥,종합분식,40,"[김밥 맛있어요, 일가면서 가끔 들리는 장소 맛도짱 입니다., 소고기2줄은 밖에서 ...","[예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 예약 없이 이용, 포장·배달...","[바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장, 바로 입장]","[일상, 나들이, 일상, 일상]","[지인・동료, 연인・배우자, 지인・동료, 연인・배우자]","['김밥']['가면', '가끔', '장소']['소고기']['사진', '실물', '매...","김밥 맛있어요일가면서 가끔 들리는 장소 맛도짱 입니다.소고기2줄은 밖에서 먹고,멸추...","['김밥', '가면', '가끔', '장소', '소고기', '사진', '실물', '매...","아침 간식여전히 일가는 도중에 들려서 몇줄 샀어요.참치, 소고기 김밥 샀는데 맛있어...","['아침', '간식', '일가', '도중', '참치', '소고기', '김밥', '오..."


In [124]:
result.to_csv('/content/drive/MyDrive/빅콘테스트/01.데이터수집 및 탐색/code/EDA/예진/data/jeju_store_keyword.csv', index=0)